[Documentation of GRPC use in Python](https://grpc.io/docs/languages/python/)

To exec the code below, you have to run the script in /tools/generate_grpc_api.sh

It will create the Python classes starting from the proto files.

To run the script, open a terminal tab and type "~/tools/generate_grpc_api.sh", the output should be:

```
(base) jovyan@9b15573c79cd:~/tools$ ~/tools/generate_grpc_api.sh
working on ptalk-ability-api
working on ptalk-channel-api
working on ptalk-nlu-api
working on ptalk-channel-grpc-api
working on ptalk-base-api
```

If you want simulate a PTalk Engine to test this code, you can run "./gradlew :ptalk-channel-api:execute"

```
[andrea@legion-rossonet-com ptalk]$ ./gradlew :ptalk-channel-api:execute

> Task :ptalk-channel-api:execute
Fake engine started, press CTRL + C to exit
<============-> 92% EXECUTING [14s]
> :ptalk-channel-api:execute
```



In [ ]:
import logging
import grpc
import sys
sys.path.insert(1, '/home/jovyan/work/grpc-apis/base')
sys.path.insert(2, '/home/jovyan/work/grpc-apis/channel')
import ptalk_channel_api_pb2_grpc as channel_service
import ptalk_channel_api_pb2 as channel_messages
import ptalk_base_api_pb2 as base_messages
import uuid
from concurrent import futures

In [ ]:
# set parameters

host = "192.168.1.59"
port_engine = 11216
port_unit = 11214
user_name = "test-user"

In [ ]:
# create stub

channel = grpc.insecure_channel(host+":"+str(port_engine))
stub = channel_service.RpcChannelCoreV1Stub(channel)

In [ ]:
# create send message function

def send(messageTxt):
    stub.CallSync(channel_messages.ChannelMessageRequest(
      channelUniqueName = user_name,
      channelUniqueMessageId = str(uuid.uuid1()),
      message = base_messages.Data(value = messageTxt)
    ))
    return "ok"

In [ ]:
# run console

# Gradio documentation: https://gradio.app/docs/

import gradio as gr
import queue

queue = queue.Queue()

with gr.Blocks() as console_channel:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        send(user_message) 
        return "", history + [[user_message, queue.get(True, 5)]]
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

In [ ]:
# class need to manage input message

class UnitChannelService(channel_service.RpcChannelUnitV1Servicer):
    def CallSync(self, request, context):
        metadata = dict(context.invocation_metadata())
        #print("METADATA ", metadata)
        #print("MESSAGE: ", request)
        queue.put(request.message.value)
        return channel_messages.ChannelMessageReply()

In [ ]:
# run channel unit service

server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
def run_unit_server():
  channel_service.add_RpcChannelUnitV1Servicer_to_server(UnitChannelService(),server)
  server.add_insecure_port("[::]:" + str(port_unit))
  server.start()
  server.wait_for_termination()

In [ ]:
# run service and Gradio console

import threading, time
thread = threading.Thread(target=run_unit_server)
thread.start()
time.sleep(1) # wait unit server start

console_channel.launch(share=True)

In [ ]:
# stop server

server.stop(1)
thread.join()
exit()